# Démonstration : Mixnets (Mix Networks)

**Objectifs** :
- Comprendre le concept de mix (mélangeur)
- Implémenter un mix de Chaum (1981)
- Comparer avec Tor (onion routing)
- Analyser les applications (vote électronique, email anonyme)

In [ ]:
from cryptography.hazmat.primitives.asymmetric import rsa, padding
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.backends import default_backend
import secrets
import time
from typing import List, Tuple

## 1. Mixnet : Principe de Base

**Problème** : Alice veut envoyer un email anonyme à Bob.

**Solution naïve (échec)** : Chiffrer et envoyer via un serveur intermédiaire.
- ❌ Serveur peut corréler entrée/sortie (timing, taille)

**Solution (Chaum 1981)** : Mix (mélangeur)

**Fonctionnement** :
1. Plusieurs utilisateurs envoient messages chiffrés au mix
2. Mix accumule un **batch** de messages
3. Mix déchiffre tous les messages
4. Mix **mélange** (permutation aléatoire)
5. Mix transmet dans ordre aléatoire

**Propriété clé** : Impossible de lier message entrant et sortant !

**Format message** : $E_{\text{PK}_{\text{mix}}}(\text{destinataire} \| \text{message})$

## 2. Implémentation d'un Mix Simple

In [ ]:
class SimpleMix:
    """
    Mix simple (Chaum 1981).
    """
    def __init__(self, name: str, batch_size: int = 5):
        self.name = name
        self.batch_size = batch_size
        
        # Générer paire de clés RSA
        self.private_key = rsa.generate_private_key(
            public_exponent=65537,
            key_size=2048,
            backend=default_backend()
        )
        self.public_key = self.private_key.public_key()
        
        # Buffer de messages entrants
        self.buffer = []
        
        print(f"🔐 Mix '{name}' initialisé (batch size: {batch_size})")
    
    def receive_message(self, encrypted_message: bytes):
        """
        Reçoit un message chiffré.
        """
        self.buffer.append(encrypted_message)
        print(f"   📨 {self.name} : Message reçu ({len(self.buffer)}/{self.batch_size})")
    
    def process_batch(self) -> List[Tuple[str, bytes]]:
        """
        Traite un batch de messages :
        1. Déchiffre tous les messages
        2. Mélange (permutation aléatoire)
        3. Retourne messages dans ordre aléatoire
        
        Returns:
            Liste de (destinataire, message)
        """
        if len(self.buffer) < self.batch_size:
            print(f"   ⏳ {self.name} : Pas assez de messages ({len(self.buffer)}/{self.batch_size})")
            return []
        
        print(f"\n🔄 {self.name} : Traitement du batch ({self.batch_size} messages)...")
        
        # Extraire batch
        batch = self.buffer[:self.batch_size]
        self.buffer = self.buffer[self.batch_size:]
        
        # Déchiffrer
        decrypted_messages = []
        for encrypted_msg in batch:
            plaintext = self.private_key.decrypt(
                encrypted_msg,
                padding.OAEP(
                    mgf=padding.MGF1(algorithm=hashes.SHA256()),
                    algorithm=hashes.SHA256(),
                    label=None
                )
            )
            
            # Format : destination_length (1 byte) | destination | message
            dest_length = plaintext[0]
            destination = plaintext[1:1+dest_length].decode('utf-8')
            message = plaintext[1+dest_length:]
            
            decrypted_messages.append((destination, message))
        
        print(f"   🔓 Déchiffrement terminé")
        
        # Mélange aléatoire
        import random
        random.shuffle(decrypted_messages)
        
        print(f"   🎲 Mélange aléatoire effectué")
        print(f"   📤 Transmission dans ordre aléatoire")
        
        return decrypted_messages

def simple_mixnet_demo():
    """
    Démonstration d'un mixnet simple.
    """
    print("=" * 70)
    print("MIXNET : Démonstration Simple")
    print("=" * 70)
    
    # Créer un mix
    print("\n🌐 Création d'un mix :")
    mix = SimpleMix("Mix Server", batch_size=5)
    
    # Plusieurs utilisateurs envoient des messages
    print("\n📝 Utilisateurs envoient des messages :")
    
    users_messages = [
        ("Alice", "Bob", b"Hello Bob from Alice"),
        ("Charlie", "Diana", b"Secret message to Diana"),
        ("Eve", "Frank", b"Data leak information"),
        ("Grace", "Heidi", b"Meeting at noon"),
        ("Ivan", "Judy", b"Project updates"),
    ]
    
    # Chiffrement et envoi
    for sender, recipient, message in users_messages:
        # Format : destination_length | destination | message
        recipient_bytes = recipient.encode('utf-8')
        plaintext = bytes([len(recipient_bytes)]) + recipient_bytes + message
        
        # Chiffrer avec clé publique du mix
        encrypted = mix.public_key.encrypt(
            plaintext,
            padding.OAEP(
                mgf=padding.MGF1(algorithm=hashes.SHA256()),
                algorithm=hashes.SHA256(),
                label=None
            )
        )
        
        print(f"   {sender} → Mix : Message pour {recipient}")
        mix.receive_message(encrypted)
        
        # Délai aléatoire (simulation)
        time.sleep(0.1)
    
    # Mix traite le batch
    print("\n⏳ Attente du batch complet...")
    output_messages = mix.process_batch()
    
    # Livraison des messages
    print(f"\n📬 Livraison des messages (ordre aléatoire) :")
    for i, (destination, message) in enumerate(output_messages, 1):
        print(f"   Message {i} → {destination} : {message}")
    
    # Analyse
    print(f"\n" + "=" * 70)
    print(f"ANALYSE DE L'ANONYMAT")
    print(f"=" * 70)
    
    print(f"\n✅ Propriétés garanties :")
    print(f"   - Mix ne peut PAS lier émetteur et destinataire")
    print(f"   - Observateur externe ne voit que trafic chiffré")
    print(f"   - Ordre de sortie aléatoire (mélange)")
    print(f"   - Batch empêche corrélation temporelle immédiate")
    
    print(f"\n⚠️  Limitations :")
    print(f"   - Mix malveillant peut tout voir (émetteur + destinataire)")
    print(f"   - Attaque statistique possible (traffic analysis long terme)")
    print(f"   - Latence élevée (attente du batch)")
    
    print(f"\n💡 Solution : Cascades de mixes (comme onion routing)")

simple_mixnet_demo()

## 3. Cascade de Mixes

**Principe** : Chaîner plusieurs mixes pour renforcer l'anonymat.

**Format message** : Chiffrement en couches (comme onion routing)

$$m_3 = E_{\text{PK}_3}(\text{destination} \| \text{message})$$
$$m_2 = E_{\text{PK}_2}(\text{Mix}_3 \| m_3)$$
$$m_1 = E_{\text{PK}_1}(\text{Mix}_2 \| m_2)$$

**Sécurité** : Tous les mixes doivent être compromis pour casser l'anonymat.

In [ ]:
def cascade_mixnet_demo():
    """
    Démonstration d'une cascade de 3 mixes.
    """
    print("\n" + "=" * 70)
    print("CASCADE DE MIXES (3 mixes)")
    print("=" * 70)
    
    # Créer 3 mixes
    print("\n🌐 Création de la cascade :")
    mix1 = SimpleMix("Mix 1", batch_size=3)
    mix2 = SimpleMix("Mix 2", batch_size=3)
    mix3 = SimpleMix("Mix 3", batch_size=3)
    
    # Messages à envoyer
    messages = [
        ("Alice", "Bob", b"Message 1"),
        ("Charlie", "Diana", b"Message 2"),
        ("Eve", "Frank", b"Message 3"),
    ]
    
    print("\n🧅 Construction des oignons (3 couches) :")
    
    for sender, recipient, message in messages:
        # Couche 3 (la plus interne)
        recipient_bytes = recipient.encode('utf-8')
        layer3 = bytes([len(recipient_bytes)]) + recipient_bytes + message
        
        encrypted_layer3 = mix3.public_key.encrypt(
            layer3,
            padding.OAEP(
                mgf=padding.MGF1(algorithm=hashes.SHA256()),
                algorithm=hashes.SHA256(),
                label=None
            )
        )
        
        # Couche 2
        mix3_addr = b"Mix 3"
        layer2 = bytes([len(mix3_addr)]) + mix3_addr + encrypted_layer3
        
        encrypted_layer2 = mix2.public_key.encrypt(
            layer2,
            padding.OAEP(
                mgf=padding.MGF1(algorithm=hashes.SHA256()),
                algorithm=hashes.SHA256(),
                label=None
            )
        )
        
        # Couche 1 (extérieure)
        mix2_addr = b"Mix 2"
        layer1 = bytes([len(mix2_addr)]) + mix2_addr + encrypted_layer2
        
        encrypted_layer1 = mix1.public_key.encrypt(
            layer1,
            padding.OAEP(
                mgf=padding.MGF1(algorithm=hashes.SHA256()),
                algorithm=hashes.SHA256(),
                label=None
            )
        )
        
        print(f"   {sender} → Mix 1 : Oignon 3-couches pour {recipient}")
        mix1.receive_message(encrypted_layer1)
    
    # Mix 1 traite le batch
    print(f"\n🔄 Mix 1 : Traitement...")
    output1 = mix1.process_batch()
    
    # Transmettre à Mix 2
    print(f"\n📤 Mix 1 → Mix 2 : Transmission de {len(output1)} messages")
    for dest, encrypted in output1:
        if dest == "Mix 2":
            mix2.receive_message(encrypted)
    
    # Mix 2 traite le batch
    print(f"\n🔄 Mix 2 : Traitement...")
    output2 = mix2.process_batch()
    
    # Transmettre à Mix 3
    print(f"\n📤 Mix 2 → Mix 3 : Transmission de {len(output2)} messages")
    for dest, encrypted in output2:
        if dest == "Mix 3":
            mix3.receive_message(encrypted)
    
    # Mix 3 traite le batch
    print(f"\n🔄 Mix 3 : Traitement...")
    output3 = mix3.process_batch()
    
    # Livraison finale
    print(f"\n📬 Livraison finale :")
    for destination, message in output3:
        print(f"   → {destination} : {message}")
    
    print(f"\n✅ Cascade de mixes : Anonymat renforcé")
    print(f"   - Il faut compromettre LES 3 mixes pour casser l'anonymat")
    print(f"   - Probabilité beaucoup plus faible")

cascade_mixnet_demo()

## 4. Comparaison : Mixnet vs Tor

In [ ]:
import pandas as pd

print("\n" + "=" * 70)
print("COMPARAISON : Mixnet vs Tor (Onion Routing)")
print("=" * 70)

comparison = pd.DataFrame({
    'Propriété': [
        'Type de trafic',
        'Latence',
        'Protection',
        'Batch processing',
        'Scalabilité',
        'Résistance traffic analysis',
        'Complexité'
    ],
    'Mixnet (Chaum 1981)': [
        'Asynchrone (email)',
        'Élevée (minutes)',
        'Forte (batching)',
        'Oui (requis)',
        'Limitée',
        'Excellente',
        'Modérée'
    ],
    'Tor (Onion Routing)': [
        'Synchrone (web)',
        'Faible (~100-300ms)',
        'Bonne',
        'Non (streaming)',
        'Excellente',
        'Modérée',
        'Élevée'
    ]
})

print()
print(comparison.to_string(index=False))

print("\n📊 Cas d'usage :")
print("\nMixnet :")
print("   ✅ Email anonyme (Mixmaster, Mixminion)")
print("   ✅ Vote électronique (anonymat + vérifiabilité)")
print("   ✅ Messages asynchrones (délai acceptable)")
print("   ❌ Navigation web (latence trop élevée)")

print("\nTor (Onion Routing) :")
print("   ✅ Navigation web anonyme")
print("   ✅ Messagerie instantanée (Signal via Tor)")
print("   ✅ Contournement de censure")
print("   ⚠️  Vote électronique (coercion possible)")

print("\n💡 Complémentarité :")
print("   - Tor : Communication temps réel")
print("   - Mixnet : Communication asynchrone haute sécurité")

## 5. Application : Vote Électronique

**Exigences** :
1. **Anonymat** : Vote secret (lien votant-vote cassé)
2. **Vérifiabilité** : Chacun peut vérifier que son vote est compté
3. **Intégrité** : Résultat correct
4. **Unicité** : Un vote par personne

**Solution avec Mixnet** :
1. Votant chiffre son vote avec clé publique du système
2. Bulletin passe par cascade de mixes
3. Mixes mélangent et déchiffrent
4. Votes anonymisés publiés sur bulletin board
5. Chacun peut vérifier le résultat

In [ ]:
print("\n" + "=" * 70)
print("APPLICATION : Vote Électronique avec Mixnet")
print("=" * 70)

print("\n🗳️  Protocole de vote :")
print("\n1️⃣ Inscription :")
print("   - Autorité électorale génère liste votants autorisés")
print("   - Chaque votant reçoit credential (token unique)")

print("\n2️⃣ Vote :")
print("   - Votant chiffre : E_pk(vote | credential)")
print("   - Envoie via cascade de mixes")
print("   - Mixes mélangent et déchiffrent progressivement")

print("\n3️⃣ Publication :")
print("   - Votes anonymisés publiés sur bulletin board public")
print("   - Format : (credential anonymisé, vote)")
print("   - Tout le monde peut vérifier le décompte")

print("\n4️⃣ Vérification :")
print("   - Votant cherche son credential sur bulletin board")
print("   - Vérifie que son vote est correct")
print("   - Vérifie que le total est correct")

print("\n✅ Garanties :")
print("   - Anonymat : Mixes cassent lien votant-vote")
print("   - Vérifiabilité : Bulletin board public")
print("   - Intégrité : Tous peuvent recompter")
print("   - Unicité : Credential unique par votant")

print("\n⚠️  Défis :")
print("   - Coercion : Votant forcé de prouver son vote")
print("   - Solution : Receipt-free voting (impossibilité de prouver)")
print("   - Vote selling : Vente de credentials")
print("   - Solution : Dépôt physique requis")

print("\n🌍 Systèmes réels :")
print("   - Helios : Vote en ligne pour élections publiques")
print("   - Scantegrity : Utilisé dans élections municipales (USA)")
print("   - Estonie : Vote électronique national (pas mixnet)")
print("   - Suisse : Pilotes e-voting cantonaux")

## 6. Attaques sur Mixnets

In [ ]:
print("\n" + "=" * 70)
print("ATTAQUES SUR MIXNETS")
print("=" * 70)

print("\n❌ 1. Flushing Attack")
print("   Scénario : Adversaire envoie nombreux messages pour forcer flush")
print("   Attaque : Isole message cible en contrôlant timing")
print("   Protection : Seuils de batch fixes, délais aléatoires")

print("\n❌ 2. (n-1) Attack")
print("   Scénario : Adversaire envoie (n-1) messages d'un batch de n")
print("   Attaque : Message cible est le n-ième, identifiable")
print("   Protection : Mixes ajoutent dummy traffic")

print("\n❌ 3. Tagging Attack")
print("   Scénario : Mix malveillant modifie ciphertext (tag)")
print("   Attaque : Suit le message tagué à travers cascade")
print("   Protection : Vérification d'intégrité (impossible avec RSA seul)")

print("\n❌ 4. Epistemic Attack (Long-term Statistics)")
print("   Scénario : Observation long terme (mois/années)")
print("   Attaque : Analyse statistique de patterns de communication")
print("   Protection : Dummy traffic, cover traffic")

print("\n❌ 5. Replay Attack")
print("   Scénario : Adversaire rejoue un message")
print("   Attaque : Peut perturber mélange, tracer")
print("   Protection : Timestamps, replay cache (seen messages)")

print("\n✅ Protections générales :")
print("   - Batch size fixe et grand (>100)")
print("   - Délais aléatoires (pool mixes)")
print("   - Dummy traffic (messages fictifs)")
print("   - Vérifiabilité (preuves zero-knowledge)")
print("   - Cascades longues (>3 mixes)")

## Conclusion

**Points clés** :
- Mixnet (Chaum 1981) : Batch processing + mélange pour anonymat
- ✅ Cascade de mixes : Sécurité renforcée (tous doivent être compromis)
- ✅ Adapté aux communications asynchrones (email, vote)
- ⚠️ Latence élevée (attente du batch)
- ⚠️ Vulnérable à attaques statistiques (long terme)

**Mixnet vs Tor** :
- Mixnet : Haute latence, forte protection, asynchrone
- Tor : Faible latence, bonne protection, synchrone
- Complémentaires selon cas d'usage

**Applications** :
- Email anonyme : Mixmaster, Mixminion (obsolètes)
- Vote électronique : Helios, Scantegrity
- Recherche active : Loopix, Katzenpost (mixnets modernes)

**Cryptographie utilisée** :
- Chiffrement : RSA-OAEP ou ElGamal
- Preuves : Zero-knowledge proofs (vérifiabilité)
- Moderne : Sphinx (format mixnet compact)